In [1]:
# Import libraries
import locale

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from datetime import datetime
from locale import atof

In [2]:
# Define function that transforms date into value corresponding to number of week in year 
# that it belongs to.
def day_to_week_of_year(date_to_transform):
    return datetime.strptime(date_to_transform + "/2017", '%d/%B/%Y').isocalendar()[1]

In [11]:
# Load data
data = pd.read_csv('train.csv', sep=';', decimal=',')

In [4]:
# See data column names.
data.columns

Index(['ID', 'TIMESTAMP', 'WEBSITE', 'GDS', 'DEPARTURE', 'ARRIVAL', 'ADULTS',
       'CHILDREN', 'INFANTS', 'TRAIN', 'HAUL_TYPE', 'DISTANCE', 'DEVICE',
       'TRIP_TYPE', 'PRODUCT', 'SMS', 'EXTRA_BAGGAGE', 'NO_GDS'],
      dtype='object')

In [12]:
# See how many missing values the dataset has.
print(data.isnull().sum())

# Since there are missing values only in variable 'DEVICE' and they are very few we'll 
# just remove them. If there were much more we could use imputation methods to fill this
# missing values.
data = data[pd.notnull(data['DEVICE'])]
data = data.reset_index(drop=True)

print(data.isnull().sum())

ID                 0
TIMESTAMP          0
WEBSITE            0
GDS                0
DEPARTURE          0
ARRIVAL            0
ADULTS             0
CHILDREN           0
INFANTS            0
TRAIN              0
HAUL_TYPE          0
DISTANCE           0
DEVICE           133
TRIP_TYPE          0
PRODUCT            0
SMS                0
EXTRA_BAGGAGE      0
NO_GDS             0
dtype: int64
ID               0
TIMESTAMP        0
WEBSITE          0
GDS              0
DEPARTURE        0
ARRIVAL          0
ADULTS           0
CHILDREN         0
INFANTS          0
TRAIN            0
HAUL_TYPE        0
DISTANCE         0
DEVICE           0
TRIP_TYPE        0
PRODUCT          0
SMS              0
EXTRA_BAGGAGE    0
NO_GDS           0
dtype: int64


In [6]:
# See distribution of classes in target variable.
data['EXTRA_BAGGAGE'].value_counts()

# Classes are imbalanced. There are much more samples belonging to the 'False' class than 
# to the 'True' class.

False    40068
True      9799
Name: EXTRA_BAGGAGE, dtype: int64

In [13]:
# Review date variables: 'TIMESTAMP', 'DEPARTURE' and 'ARRIVAL'
print(data['TIMESTAMP'][0:5])
print(data['DEPARTURE'][0:5])
print(data['ARRIVAL'][0:5])

# All three date variables have the same format 'day_number/month_name'.
# We'll transform these values to numeric values corresponding to number 
# of week in year that the date belongs to.

data['TIMESTAMP'] = data['TIMESTAMP'].apply(day_to_week_of_year)
data['DEPARTURE'] = data['DEPARTURE'].apply(day_to_week_of_year)
data['ARRIVAL'] = data['ARRIVAL'].apply(day_to_week_of_year)

print(data['TIMESTAMP'][0:5])
print(data['DEPARTURE'][0:5])
print(data['ARRIVAL'][0:5])

0    01/July
1    01/July
2    01/July
3    01/July
4    01/July
Name: TIMESTAMP, dtype: object
0      22/July
1      29/July
2      29/July
3      24/July
4    11/August
Name: DEPARTURE, dtype: object
0      25/July
1      29/July
2    19/August
3    04/August
4    11/August
Name: ARRIVAL, dtype: object
0    26
1    26
2    26
3    26
4    26
Name: TIMESTAMP, dtype: int64
0    29
1    30
2    30
3    30
4    32
Name: DEPARTURE, dtype: int64
0    30
1    30
2    33
3    31
4    32
Name: ARRIVAL, dtype: int64


In [ ]:
# Transform string variables 'WEBSITE', 'DEVICE', 'HAUL_TYPE', 'TRIP_TYPE' and 'PRODUCT' 
# to categorical using LabelEncoder.
# IMPORTANT: form 'WEBSITE' variable, we are not sure, all levels that this categorical 
# variable can take, arein the train.csv dataset.
label_encoder = LabelEncoder()

print(data['WEBSITE'].dtype)
data['WEBSITE'] = pd.Series(label_encoder.fit_transform(data['WEBSITE'])).values
print(data['WEBSITE'][0:5])

print(data['DEVICE'].dtype)
data['DEVICE'] = pd.Series(label_encoder.fit_transform(data['DEVICE'])).values
print(data['DEVICE'][0:5])

print(data['HAUL_TYPE'].dtype)
data['HAUL_TYPE'] = pd.Series(label_encoder.fit_transform(data['HAUL_TYPE'])).values
print(data['HAUL_TYPE'][0:5])

print(data['TRIP_TYPE'].dtype)
data['TRIP_TYPE'] = pd.Series(label_encoder.fit_transform(data['TRIP_TYPE'])).values
print(data['TRIP_TYPE'][0:5])

print(data['PRODUCT'].dtype)
data['PRODUCT'] = pd.Series(label_encoder.fit_transform(data['PRODUCT'])).values
print(data['PRODUCT'][0:5])

In [14]:
# Transform string variables 'WEBSITE', 'DEVICE', 'HAUL_TYPE', 'TRIP_TYPE' and 'PRODUCT' 
# to categorical using LabelEncoder.
# IMPORTANT: form 'WEBSITE' variable, we are not sure, all levels that this categorical 
# variable can take, arein the train.csv dataset.

# Won't binarize 'WEBSITE' variable because it has too many levels.
label_encoder = LabelEncoder()

print(data['WEBSITE'].dtype)
data['WEBSITE'] = pd.Series(label_encoder.fit_transform(data['WEBSITE'])).values
print(data['WEBSITE'][0:5])

label_encoder = LabelBinarizer()

print(data['DEVICE'].dtype)
print(data['DEVICE'].shape)
print(data['DEVICE'][0:5])
encoder_result = label_encoder.fit_transform(data['DEVICE'])
device_columns = ["DEVICE_" + str(bin_class) for bin_class in label_encoder.classes_]
data_device = pd.DataFrame(encoder_result, columns=device_columns)
print("data device shape:", data_device.shape)
print(data_device[0:5])

print("data shape:", data.shape)
print("new device columns:", len(device_columns))
data = data.drop('DEVICE', 1)
data = data.reset_index(drop=True)
print("data shape:", data.shape)
data = pd.concat([data, data_device], axis=1)
data = data.reset_index(drop=True)
print("new data shape:", data.shape)
print(data[device_columns][0:5])
print()

print(data['HAUL_TYPE'].dtype)
print(data['HAUL_TYPE'].shape)
print(data['HAUL_TYPE'][0:5])
encoder_result = label_encoder.fit_transform(data['HAUL_TYPE'])
haul_type_columns = ["HAUL_TYPE_" + str(bin_class) for bin_class in label_encoder.classes_]
data_haul_type = pd.DataFrame(encoder_result, columns=haul_type_columns)
print("data haul_type shape:", data_haul_type.shape)
print(data_haul_type[0:5])

print("data shape:", data.shape)
print("new haul_type columns:", len(haul_type_columns))
data = data.drop('HAUL_TYPE', 1)
data = data.reset_index(drop=True)
print("data shape:", data.shape)
data = pd.concat([data, data_haul_type], axis=1)
data = data.reset_index(drop=True)
print("new data shape:", data.shape)
print(data[haul_type_columns][0:5])
print()

print(data['TRIP_TYPE'].dtype)
print(data['TRIP_TYPE'].shape)
print(data['TRIP_TYPE'][0:5])
encoder_result = label_encoder.fit_transform(data['TRIP_TYPE'])
trip_type_columns = ["TRIP_TYPE_" + str(bin_class) for bin_class in label_encoder.classes_]
data_trip_type = pd.DataFrame(encoder_result, columns=trip_type_columns)
print("data trip_type shape:", data_trip_type.shape)
print(data_trip_type[0:5])

print("data shape:", data.shape)
print("new trip_type columns:", len(trip_type_columns))
data = data.drop('TRIP_TYPE', 1)
data = data.reset_index(drop=True)
print("data shape:", data.shape)
data = pd.concat([data, data_trip_type], axis=1)
data = data.reset_index(drop=True)
print("new data shape:", data.shape)
print(data[trip_type_columns][0:5])
print()

print(data['PRODUCT'].dtype)
print(data['PRODUCT'].shape)
print(data['PRODUCT'][0:5])
encoder_result = label_encoder.fit_transform(data['PRODUCT'])
data_product = pd.Series(encoder_result[:, 0], name='PRODUCT')
print("data product shape:", data_product.shape)
print(data_product[0:5])

print("data shape:", data.shape)
data = data.drop('PRODUCT', 1)
data = data.reset_index(drop=True)
print("data shape:", data.shape)
data = pd.concat([data, data_product], axis=1)
data = data.reset_index(drop=True)
print("new data shape:", data.shape)
print(data['PRODUCT'][0:5])
print()

object
0    11
1    18
2    54
3    50
4    11
Name: WEBSITE, dtype: int64
object
(49867,)
0        TABLET
1    SMARTPHONE
2        TABLET
3    SMARTPHONE
4      COMPUTER
Name: DEVICE, dtype: object
data device shape: (49867, 4)
   DEVICE_COMPUTER  DEVICE_OTHER  DEVICE_SMARTPHONE  DEVICE_TABLET
0                0             0                  0              1
1                0             0                  1              0
2                0             0                  0              1
3                0             0                  1              0
4                1             0                  0              0
data shape: (49867, 18)
new device columns: 4
data shape: (49867, 17)
new data shape: (49867, 21)
   DEVICE_COMPUTER  DEVICE_OTHER  DEVICE_SMARTPHONE  DEVICE_TABLET
0                0             0                  0              1
1                0             0                  1              0
2                0             0                  0              1
3  

In [ ]:
# ['ID', 'TIMESTAMP', 'WEBSITE', 'GDS', 'DEPARTURE', 'ARRIVAL', 'ADULTS',
#  'CHILDREN', 'INFANTS', 'TRAIN', 'HAUL_TYPE', 'DISTANCE', 'DEVICE',
#  'TRIP_TYPE', 'PRODUCT', 'SMS', 'EXTRA_BAGGAGE', 'NO_GDS']

# Check all integer variables 'GDS', 'NO_GDS', 'ADULTS', 'CHILDREN' and 'INFANTS', have 
# their corresponding dtype in the pandas DataFrame.
print(data['GDS'].dtype)
print(data['NO_GDS'].dtype)
print(data['ADULTS'].dtype)
print(data['CHILDREN'].dtype)
print(data['INFANTS'].dtype)

In [ ]:
# Check all boolean variables 'TRAIN', 'SMS' and 'EXTRA_BAGGAGE' have their corresponding 
# dtype in the pandas DataFrame.
print(data['TRAIN'].dtype)
print(data['SMS'].dtype)
print(data['EXTRA_BAGGAGE'].dtype)

In [ ]:
# Check float variable 'DISTANCE' has it's corresponding dtype in the pandas DataFrame.
print(data['DISTANCE'].dtype)

In [15]:
# Drop 'ID' variable since it is useless.
data = data.drop('ID', 1)
data = data.reset_index(drop=True)

In [16]:
# Store 'clean' dataset into new .csv file
# data.to_csv('clean_train.csv', index=False, sep=';')
data.to_csv('clean_binarized_train.csv', index=False, sep=';')